# RabbitMQ Basics with Docker: A Hands-On Tutorial

## Introduction

RabbitMQ is a robust messaging broker that allows applications to communicate asynchronously. It supports a variety of messaging protocols and is widely used for managing message queues between distributed systems.

In this tutorial, you will:
- Set up RabbitMQ using Docker.
- Interact with RabbitMQ using Python to send and receive messages.
- Understand basic concepts like producers, consumers, and queues.

## Prerequisites

Ensure that Docker is installed on your system. You can install Docker by following [this guide](https://docs.docker.com/get-docker/).

Additionally, ensure Python is installed with the `pika` library, which is used to interact with RabbitMQ.

Install `pika`:

In [1]:
%pip install pika

  Using cached pika-1.3.2-py3-none-any.whl.metadata (13 kB)
Using cached pika-1.3.2-py3-none-any.whl (155 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Let's ensure that pika is installed correctly. No error messages means that everything is okay.

In [2]:
import pika

## Setting Up RabbitMQ Server Using Docker

We will use Docker to set up a RabbitMQ server, which is easy and avoids any manual installation.

### Pull RabbitMQ Docker Image

We’ll start by pulling the official RabbitMQ Docker image.

In [3]:
!docker pull rabbitmq:3-management

3-management: Pulling from library/rabbitmq

3066d3d5: Pulling fs layer 
4a386d13: Pulling fs layer 
1af0856e: Pulling fs layer 
c57eeafe: Pulling fs layer 
7ac6489f: Pulling fs layer 
1c409bb2: Pulling fs layer 
860a5545: Pulling fs layer 
0d7bf8ba: Pulling fs layer 
72a0a4d4: Pulling fs layer 
Digest: sha256:ad6939bab2827c50591f3ad0345f61c6ec00a8980723d3e9983cf51d5c485103
Status: Downloaded newer image for rabbitmq:3-management
docker.io/library/rabbitmq:3-management


The `3-management` tag includes the RabbitMQ Management Plugin, allowing you to monitor and manage the RabbitMQ server through a web interface.

### Run the RabbitMQ Container

Now, let's run RabbitMQ in a Docker container. We will expose two ports:
- **5672**: The default port for RabbitMQ.
- **15672**: The web management interface.

In [4]:
!docker run -d --name rabbitmq-server -p 5672:5672 -p 15672:15672 rabbitmq:3-management

e75234ffe2393d021ea23f03d894eabb85e23af7f303031498da1238d4f71932


### Access the RabbitMQ Management Interface

After running the above command, RabbitMQ will be accessible through the web interface. Open a browser and visit:
[http://localhost:15672](http://localhost:15672)

Use the default credentials to log in:
- Username: `guest`
- Password: `guest`

## Understanding RabbitMQ Concepts

Before we jump into code, let's review the core RabbitMQ concepts:

- **Producer**: A producer sends messages to RabbitMQ.
- **Consumer**: A consumer receives messages from RabbitMQ.
- **Queue**: A buffer that stores messages until they are consumed.
- **Exchange**: A mechanism that routes messages to one or more queues based on rules.

## Sending Messages to RabbitMQ (Producer)

Now that the RabbitMQ server is up and running, let's write a Python script to send a message to a queue.

First, we'll import the necessary libraries:

In [8]:
import pika

### Establish Connection to RabbitMQ

We need to establish a connection to the RabbitMQ server running on `localhost`.

In [9]:
def get_connection():
    connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
    return connection


### Sending a Message to a Queue

Let's write a function to send a message to a queue called `hello`.

In [10]:
def send_message(message):
    connection = get_connection()
    channel = connection.channel()

    # Declare a queue
    channel.queue_declare(queue='hello')

    # Publish a message to the queue
    channel.basic_publish(exchange='',
                          routing_key='hello',
                          body=message)
    print(f" [x] Sent {message}")
    connection.close()

# Test sending a message
send_message("Hello from RabbitMQ!")

 [x] Sent Hello from RabbitMQ!


After a few seconds, you should see some stats appear on the RabbitMQ dashboard under 'Totals', indicating that messages were sent.

**Explanation**:
- **`queue_declare`**: This ensures that the queue exists before we send a message to it. If the queue doesn’t exist, it will be created.
- **`basic_publish`**: This sends the message to the `hello` queue.

## Receiving Messages from RabbitMQ (Consumer)

Now let's create a consumer that listens for messages from the `hello` queue.

### Defining a Consumer

In [11]:
def receive_message():
    connection = get_connection()
    channel = connection.channel()

    # Declare the same queue
    channel.queue_declare(queue='hello')

    # Callback function to process messages
    def callback(ch, method, properties, body):
        print(f" [x] Received {body}")

    # Tell RabbitMQ to deliver messages from the queue
    channel.basic_consume(queue='hello',
                          on_message_callback=callback,
                          auto_ack=True)

    print(' [*] Waiting for messages. To exit press Interrupt (the square at the top of the notebook)')
    channel.start_consuming()

# Start receiving messages
receive_message()

 [*] Waiting for messages. To exit press Interrupt (the square at the top of the notebook)
 [x] Received b'Hello from RabbitMQ!'
 [x] Received b'Hello from RabbitMQ!'


KeyboardInterrupt: 

## Example producer script

In [12]:
%%writefile send.py
import pika
def get_connection():
    connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
    return connection

def send_message(message):
    connection = get_connection()
    channel = connection.channel()

    # Declare a queue
    channel.queue_declare(queue='hello')

    # Publish a message to the queue
    channel.basic_publish(exchange='',
                          routing_key='hello',
                          body=message)
    print(f" [x] Sent {message}")
    connection.close()

# Test sending a message
send_message("Hello from RabbitMQ!")

Overwriting send.py


## Sending and Receiving Messages Together

Now that we have both producer and consumer code, let's see them working together:
1. Run the consumer in the next cell.
2. Open a terminal in the current folder, and run `python send.py`. Repeat as you wish, to see the multiple messages arriving. Note: Messages may take a few seconds to arrive and be printed.

In [13]:
# Start receiving messages
receive_message()

 [*] Waiting for messages. To exit press Interrupt (the square at the top of the notebook)


KeyboardInterrupt: 

## RabbitMQ Web Management Interface

You can monitor the status of queues, connections, and messages through the RabbitMQ Management Interface:

- Go to http://localhost:15672
- Log in with the default credentials (`guest`/`guest`).
- You can see details about the `hello` queue and the messages being sent and received.

## Stopping the RabbitMQ Docker Container

After you're done with RabbitMQ, you can stop and remove the Docker container:

In [14]:
!docker stop rabbitmq-server
!docker rm rabbitmq-server

rabbitmq-server
rabbitmq-server




This will stop the RabbitMQ server and remove the container to free up resources.
